In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd

# Carga tu dataframe (aquí asumo que el archivo está en formato CSV)
df = pd.read_csv("imdb_ds_2k_clean.csv")  # Cambia esto por la ruta a tu archivo
df

In [2]:
reviews = df["sw_text"].tolist()  # Extrae la columna de reviews como lista de oraciones

In [ ]:
# Inicializa el tokenizador y modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Función para generar embeddings de una palabra
def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Promedia los embeddings para los tokens de la palabra
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    # Reduce la dimensión de 768 a 300
    embeddings = embeddings[:300] if embeddings.shape[0] > 300 else np.pad(embeddings, (0, 300 - embeddings.shape[0]), 'constant')
    return embeddings

# Prepara un conjunto único de palabras en todas las reviews
unique_words = set()
for review in reviews:
    words = review.split()
    unique_words.update(words)

# Abre un archivo para escribir los embeddings
with open("embedding_imdb2k_bert.txt", "w") as f:
    # Escribe el encabezado con el número de palabras y la dimensión
    f.write(f"{len(unique_words)} 300\n")
    
    for word in unique_words:
        # Obtén el embedding de la palabra
        embedding = get_word_embedding(word)
        # Convierte el embedding en una cadena de texto
        embedding_str = " ".join(map(str, embedding))
        # Escribe la palabra seguida de su embedding
        f.write(f"{word} {embedding_str}\n")

print("Embeddings guardados en 'word_embeddings.txt'") #5min
